[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nilsjennissen/machine-learning/blob/main/notebooks/classification_models.ipynb)

# Classification Models in Practice
###

This notebook presents a practical approach to building and evaluating classification models, using decision trees and random forests as examples. The project aims to provide a step-by-step guide to working with these algorithms, from importing and preprocessing data to fitting and tuning models. The notebook also offers tips on how to visualize decision trees and interpret their results.

The first sections of the notebook focus on decision trees, which are a simple yet powerful algorithm that can be used for classification and regression tasks. The code shows how to train decision trees on a sample dataset and how to visualize their structure using the Graphviz library. The notebook then moves on to random forests, which are an ensemble of decision trees that can improve performance and reduce overfitting.

One important aspect of building classification models is hyperparameter tuning, which involves selecting the best values for algorithm-specific parameters that control model complexity and accuracy. The notebook includes an example of using GridSearchCV to search for the best hyperparameters for a random forest model.

The code also shows how to evaluate model performance using various metrics, such as accuracy, precision, recall, and F1 score.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Common imports
import numpy as np
import os

In [24]:
% matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

# Where to save the figures (adjust this to your convenience)
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)


UsageError: Line magic function `%` not found.


In [25]:
df = pd.read_csv("../data/hr_employee_attrition.csv")
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [27]:
df.drop(['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'], axis="columns", inplace=True)

categorical_col = []
for column in df.columns:
    if df[column].dtype == object and len(df[column].unique()) <= 50:
        categorical_col.append(column)

df['Attrition'] = df.Attrition.astype("category").cat.codes

In [28]:
categorical_col.remove('Attrition')

In [29]:
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
for column in categorical_col:
    df[column] = label.fit_transform(df[column])

In [30]:
df=pd.get_dummies(df, columns=categorical_col,drop_first=True)

In [31]:
from sklearn.model_selection import train_test_split

X = df.drop('Attrition', axis=1)
y = df.Attrition

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [32]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")

    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [33]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(  random_state=42)
tree_clf.fit(X_train, y_train)

print_score(tree_clf, X_train, y_train, X_test, y_test, train=True)
print_score(tree_clf, X_train, y_train, X_test, y_test, train=False)

Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
               0      1  accuracy  macro avg  weighted avg
precision    1.0    1.0       1.0        1.0           1.0
recall       1.0    1.0       1.0        1.0           1.0
f1-score     1.0    1.0       1.0        1.0           1.0
support    853.0  176.0       1.0     1029.0        1029.0
_______________________________________________
Confusion Matrix: 
 [[853   0]
 [  0 176]]

Test Result:
Accuracy Score: 79.82%
_______________________________________________
CLASSIFICATION REPORT:
                    0          1  accuracy   macro avg  weighted avg
precision    0.892183   0.300000  0.798186    0.596092      0.810271
recall       0.871053   0.344262  0.798186    0.607657      0.798186
f1-score     0.881491   0.320611  0.798186    0.601051      0.803909
support    380.000000  61.000000  0.798186  441.000000    441.000000
_______________________________________________
Conf

## GridSearch

In [34]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth': range(2, 10), 'min_samples_split': range(2, 10), 'min_samples_leaf': range(2, 10)}
grid_search_cv = GridSearchCV(tree_clf, parameters, n_jobs=-1, verbose=1, cv=3)
grid_search_cv.fit(X_train, y_train)


Fitting 3 folds for each of 512 candidates, totalling 1536 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             verbose=1)

In [35]:
grid_search_cv.best_params_

{'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 2}

In [36]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_depth=2, min_samples_leaf=2, random_state=42)

In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {
    "criterion":("gini", "entropy"),
    "splitter":("best", "random"),
    "max_depth":(list(range(1, 20))),
    "min_samples_split":[2, 3, 4],
    "min_samples_leaf":list(range(1, 20)),
    "class_weight":("balanced",None)
}

In [38]:
tree_clf = DecisionTreeClassifier(random_state=42)
tree_cv = GridSearchCV(tree_clf, params, scoring="accuracy", n_jobs=-1, verbose=1, cv=3)
tree_cv.fit(X_train, y_train)
best_params = tree_cv.best_params_
print(f"Best paramters: {best_params})")

tree_clf = DecisionTreeClassifier(**best_params)
tree_clf.fit(X_train, y_train)
print_score(tree_clf, X_train, y_train, X_test, y_test, train=True)
print_score(tree_clf, X_train, y_train, X_test, y_test, train=False)

Fitting 3 folds for each of 8664 candidates, totalling 25992 fits
Best paramters: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 10, 'min_samples_split': 2, 'splitter': 'best'})
Train Result:
Accuracy Score: 88.34%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy    macro avg  weighted avg
precision    0.892819    0.791667  0.883382     0.842243      0.875518
recall       0.976553    0.431818  0.883382     0.704186      0.883382
f1-score     0.932811    0.558824  0.883382     0.745817      0.868844
support    853.000000  176.000000  0.883382  1029.000000   1029.000000
_______________________________________________
Confusion Matrix: 
 [[833  20]
 [100  76]]

Test Result:
Accuracy Score: 85.26%
_______________________________________________
CLASSIFICATION REPORT:
                    0          1  accuracy   macro avg  weighted avg
precision    0.883212   0.433333  0.852608    0.658273 

## Visualize

In [39]:
# Visualize the Decision Tree (you need to have graphviz installed)
from graphviz import Source
from sklearn.tree import export_graphviz


features = list(df.columns)
features.remove("Attrition")

# Call the export with Graphviz
export_graphviz(
    tree_clf,
    out_file=os.path.join(IMAGES_PATH, "tree.dot"),
    feature_names=features,
    rounded=True,
    filled=True
)

# Load the file in the notebook
Source.from_file(os.path.join(IMAGES_PATH, "tree.dot"))


NameError: name 'IMAGES_PATH' is not defined

In [56]:
from sklearn.ensemble import RandomForestClassifier

n_estimators = [100, 10, 200, 1000, 1500]
class_weight = ("balanced",None)


params_grid = {'n_estimators': n_estimators,  'class_weight':class_weight}

rf_clf = RandomForestClassifier(random_state=42)

rf_cv = GridSearchCV(rf_clf, params_grid, scoring="accuracy", cv=3, verbose=2, n_jobs=-1)


rf_cv.fit(X_train, y_train)
best_params = rf_cv.best_params_
print(f"Best parameters: {best_params}")

rf_clf = rf_cv.best_estimator_

rf_clf.fit(X_train, y_train)

print_score(rf_clf, X_train, y_train, X_test, y_test, train=True)
print_score(rf_clf, X_train, y_train, X_test, y_test, train=False)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters: {'class_weight': None, 'n_estimators': 1500}
Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
               0      1  accuracy  macro avg  weighted avg
precision    1.0    1.0       1.0        1.0           1.0
recall       1.0    1.0       1.0        1.0           1.0
f1-score     1.0    1.0       1.0        1.0           1.0
support    853.0  176.0       1.0     1029.0        1029.0
_______________________________________________
Confusion Matrix: 
 [[853   0]
 [  0 176]]

Test Result:
Accuracy Score: 86.39%
_______________________________________________
CLASSIFICATION REPORT:
                    0          1  accuracy   macro avg  weighted avg
precision    0.870370   0.555556  0.863946    0.712963      0.826825
recall       0.989474   0.081967  0.863946    0.535720      0.863946
f1-score     0.926108   0.142857  0.863946    0.534483      0.81776

In [57]:
for name, score in zip(df.columns, rf_clf.feature_importances_):
    print(name, score)

Age 0.06373871284470348
Attrition 0.04922563099778
DailyRate 0.042598011500189964
DistanceFromHome 0.019424930753330762
Education 0.024934072324282967
EnvironmentSatisfaction 0.04343276633156279
HourlyRate 0.020557703614181643
JobInvolvement 0.02015790808528315
JobLevel 0.024395070145836525
JobSatisfaction 0.07783915314609338
MonthlyIncome 0.04690310732258046
MonthlyRate 0.03724306562433447
NumCompaniesWorked 0.03307826279643549
PercentSalaryHike 0.004455636134686059
PerformanceRating 0.022903826239281482
RelationshipSatisfaction 0.03272692852446201
StockOptionLevel 0.05130994197548143
TotalWorkingYears 0.02536204231145577
TrainingTimesLastYear 0.020170683033380845
WorkLifeBalance 0.04433240454724507
YearsAtCompany 0.03215279109119396
YearsInCurrentRole 0.024822878098827043
YearsSinceLastPromotion 0.03257365709054904
YearsWithCurrManager 0.012724243903703652
BusinessTravel_1 0.007727338755088596
BusinessTravel_2 0.008435949283968189
Department_1 0.0087979127150551
Department_2 0.008049

# Test

In [54]:
from matplotlib.colors import ListedColormap

def plot_decision_boundary(clf, X, y, axes=[0, 7.5, 0, 3], iris=True, legend=False, plot_training=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if not iris:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    if plot_training:
        plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", label="Iris setosa")
        plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", label="Iris versicolor")
        plt.plot(X[:, 0][y==2], X[:, 1][y==2], "g^", label="Iris virginica")
        plt.axis(axes)
    if iris:
        plt.xlabel("Petal length", fontsize=14)
        plt.ylabel("Petal width", fontsize=14)
    else:
        plt.xlabel(r"$x_1$", fontsize=18)
        plt.ylabel(r"$x_2$", fontsize=18, rotation=0)
    if legend:
        plt.legend(loc="lower right", fontsize=14)

In [55]:
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
save_fig()